In [1]:
import pandas as pd
import urllib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import time
from PIL import Image

import torch

use_cuda = torch.cuda.is_available()



In [2]:
for dirname, _, filenames in os.walk('/Downloads/LPR'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
df = pd.read_json("Indian_Number_plates.json", lines=True)
df.head()

,content,annotation,extras
0,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
1,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
2,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
3,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
4,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN


In [4]:
os.mkdir("Indian Number Plates")

In [5]:
dataset = dict()
dataset["image_name"] = list()
dataset["image_width"] = list()
dataset["image_height"] = list()
dataset["top_x"] = list()
dataset["top_y"] = list()
dataset["bottom_x"] = list()
dataset["bottom_y"] = list()

counter = 0
for index, row in df.iterrows():
    img = urllib.request.urlopen(row["content"])
    img = Image.open(img)
    img = img.convert('RGB')
    img.save("Indian Number Plates/licensed_car{}.jpeg".format(counter), "JPEG")
    
    dataset["image_name"].append("licensed_car{}".format(counter))
    
    data = row["annotation"]
    
    dataset["image_width"].append(data[0]["imageWidth"])
    dataset["image_height"].append(data[0]["imageHeight"])
    dataset["top_x"].append(data[0]["points"][0]["x"])
    dataset["top_y"].append(data[0]["points"][0]["y"])
    dataset["bottom_x"].append(data[0]["points"][1]["x"])
    dataset["bottom_y"].append(data[0]["points"][1]["y"])
    
    counter += 1
print("Downloaded {} car images.".format(counter))

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
df = pd.DataFrame(dataset)
df.head()

In [ ]:
df.to_csv("indian_license_plates.csv", index=False)

In [ ]:
df["image_name"] = df["image_name"] + ".jpeg"
df.drop(["image_width", "image_height"], axis=1, inplace=True)
df.head()

In [ ]:
lucky_test_samples = np.random.randint(0, len(df), 5)
reduced_df = df.drop(lucky_test_samples, axis=0)


In [ ]:
WIDTH = 224
HEIGHT = 224
CHANNEL = 3

def show_img(index):
    image = cv2.imread("Indian Number Plates/" + df["image_name"].iloc[index])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, dsize=(WIDTH, HEIGHT))

    tx = int(df["top_x"].iloc[index] * WIDTH)
    ty = int(df["top_y"].iloc[index] * HEIGHT)
    bx = int(df["bottom_x"].iloc[index] * WIDTH)
    by = int(df["bottom_y"].iloc[index] * HEIGHT)

    image = cv2.rectangle(image, (tx, ty), (bx, by), (0, 0, 255), 1)
    plt.imshow(image)
    plt.show()

In [ ]:
show_img(1)

In [ ]:
from torchvision import datasets
from PIL import ImageFile
import torchvision.transforms as transforms


### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
data = "data"
batch_size = 64

train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomRotation(45),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.CenterCrop(256),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])
test_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(256),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])
valid_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(256),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])
trainset = datasets.ImageFolder(data+"/train",train_transform)
testset = datasets.ImageFolder(data+"/test",test_transform)
validset = datasets.ImageFolder(data+"/valid",valid_transform)

ImageFile.LOAD_TRUNCATED_IMAGES = True
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv1 = nn.Conv2d(3,96,kernel_size = 10,stride =4,padding = 1)
        self.bn1   = nn.BatchNorm2d(96)
        self.conv2 = nn.Conv2d(96,192,kernel_size = 5,stride =1,padding = 1)
        self.bn2   = nn.BatchNorm2d(192)
        self.conv3 = nn.Conv2d(192,384,kernel_size = 3,stride =1,padding = 1)
        self.bn3   = nn.BatchNorm2d(384)
        #self.conv4 = nn.Conv2d(384,384,kernel_size = 3,stride =1,padding = 1)
        #self.bn4   = nn.BatchNorm2d(384)
        #self.conv5 = nn.Conv2d(384,576,kernel_size = 3,stride =1,padding = 1)
        #self.bn5   = nn.BatchNorm2d(576)
        #self.conv6 = nn.Conv2d(576,128,kernel_size = 3,stride =1,padding = 1)


        self.pool = nn.MaxPool2d(3,2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((5,5))
        
        self.fc1 = nn.Linear(384*5*5,500)
        #self.fc2 = nn.Linear(1000,500)
        self.fc3 = nn.Linear(500,2)
        
        self.dp= nn.Dropout(0.3)
        self.dp1= nn.Dropout(0.2)


        

    def forward(self, x):
        ## Define forward behavior
        x=self.pool(self.bn1((F.relu(self.conv1(x)))))
        x=self.pool(self.bn2((F.relu(self.conv2(x)))))
        x=self.bn3(F.relu(self.conv3(x)))
        #x=self.bn4(F.relu(self.conv4(x)))
        #x=self.pool(self.bn5(self.bn5(F.relu(self.conv5(x)))))
        #x=self.pool(F.relu(self.conv6(x)))

        x=self.avgpool(x)
        x = x.view(-1, 384 * 5 * 5)
        x=self.dp(F.relu(self.fc1(x)))
        #x=self.dp1(F.relu(self.fc2(x)))
        x= self.fc3(x)
        
                    

                    
       
        return x
print(Net)
#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()
print(model_scratch)


# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    criterion_scratch = criterion_scratch.cuda()

### TODO: select optimizer
optimizer_scratch = optim.Adadelta(model_scratch.parameters(),rho = 0.8)

In [ ]:
def train(n_epochs,model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                model = model.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data)
            loss =criterion(output,target)
            loss.backward()
            optimizer.step()
            ## record the average training loss, using something like
            #train_loss += loss.item()*data.size(0)
            #train_loss = train_loss/len(trainset)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
    
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(valid_loader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                model = model.cuda()
            ## update the average validation loss
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output,target)
            #valid_loss += loss.item()*data.size(0)
            #valid_loss = valid_loss/len(validset)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

            

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

    # return trained model
    return model


# train the model
model_scratch = train(10, model_scratch, optimizer_scratch, criterion_scratch, use_cuda, 'model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))


In [ ]:
def test( model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(test_loader):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(model_scratch, criterion_scratch, use_cuda)